In [1]:
from ipywidgets import VBox, HBox
from pyspark.sql import SparkSession
from perspective import PerspectiveWidget

from helpers.data import (
    HOST,
    MACHINES_PORT,
    USAGE_PORT,
    STATUS_PORT,
    JOBS_PORT,
)
from helpers.spark import (
    MACHINE_SCHEMA,
    MACHINE_SCHEMA_SPARK,
    USAGE_SCHEMA,
    USAGE_SCHEMA_SPARK,
    STATUS_SCHEMA,
    STATUS_SCHEMA_SPARK,
    JOBS_SCHEMA,
    JOBS_SCHEMA_SPARK,
)

In [2]:
# Important imports
from helpers.spark import (
    get_df_from_server,
    push_to_perspective,
)
from helpers.fastapi import (
    perspective_spark_bridge,
    start_server,
)

In [3]:
spark = SparkSession.builder.appName("Perspective Demo").getOrCreate()

24/06/07 15:35:29 WARN Utils: Your hostname, mbp-m1 resolves to a loopback address: 127.0.0.1; using 10.0.1.10 instead (on interface en0)
24/06/07 15:35:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/07 15:35:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/07 15:35:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Get spark streaming dfs
machines_df = get_df_from_server(spark, MACHINE_SCHEMA_SPARK, HOST, MACHINES_PORT)
usage_df = get_df_from_server(spark, USAGE_SCHEMA_SPARK, HOST, USAGE_PORT)
status_df = get_df_from_server(spark, STATUS_SCHEMA_SPARK, HOST, STATUS_PORT)
jobs_df = get_df_from_server(spark, JOBS_SCHEMA_SPARK, HOST, JOBS_PORT)

24/06/07 15:35:30 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/06/07 15:35:31 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/06/07 15:35:31 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/06/07 15:35:31 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
# construct 4 separate perspective widgets. Each will have its own table internally
machines_widget = PerspectiveWidget(MACHINE_SCHEMA, index="machine_id", settings=False)
usage_widget = PerspectiveWidget(USAGE_SCHEMA, index="machine_id", settings=False)
status_widget = PerspectiveWidget(STATUS_SCHEMA, index="machine_id", sort=[["last_update", "desc"]], settings=False)
jobs_widget = PerspectiveWidget(JOBS_SCHEMA, sort=[["start_time", "desc"]], settings=False)

In [6]:
# a little bit of layout with ipywidgets
VBox(children=[
    HBox(children=[machines_widget, usage_widget]),
    HBox(children=[status_widget, jobs_widget]),
])

In [7]:
# %%capture
app = perspective_spark_bridge(
    ["machines"],
    [machines_widget],
)
port = start_server(app)

CRITICAL:root:Listening on http://localhost:50154
INFO:     Started server process [6872]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:50154 (Press CTRL+C to quit)


In [8]:
push_to_perspective(machines_df, "machines", "localhost", port)

24/06/07 14:59:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pm/0nt85wh51h39f604mgmyyfwm0000gn/T/temporary-1ffec542-273e-4b0b-be94-2e2de1a69c7f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/07 14:59:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/07 14:59:13 WARN TextSocketMicroBatchStream: Stream closed by localhost:8083
